In [1]:
import numpy as np

import time
import mujoco
import mujoco.viewer
from scipy.linalg import block_diag
import mujoco
import numpy as np
import matplotlib.pyplot as plt
import mujoco.viewer

import mediapy as media
import time

import matplotlib.pyplot as plt
from scipy.linalg import null_space
from scipy.spatial.transform import Rotation as R

In [2]:
from main_redundancy_sim import LeapNodeMujoco, GraspClass, PosRot, GradientDescentIK, OnlyPosIK

model_path = '/home/iitgn-robotics/Saniya/redundancy-leap/leap-mujoco/model/leap hand/leaphand_redundancy.xml'
leap_hand = LeapNodeMujoco(model_path)

IK_index=OnlyPosIK('/home/iitgn-robotics/Saniya/redundancy-leap/leap-mujoco/model/leap hand/redundancy/0_index_sim.xml')
IK_thumb=OnlyPosIK('/home/iitgn-robotics/Saniya/redundancy-leap/leap-mujoco/model/leap hand/redundancy/0_thumb_sim.xml')
# IK_thumb_pos=OnlyPosIK('/home/sysidea/leap_hand_mujoco/model/leap hand/thumb.xml')
# IK_middle=OnlyPosIK('/home/iitgn-robotics/Saniya/redundancy-leap/leap-mujoco/model/leap hand/middle.xml')
# IK_tertiary=OnlyPosIK('/home/iitgn-robotics/Saniya/redundancy-leap/leap-mujoco/model/leap hand/tertiary.xml')

In [3]:
camera1 = mujoco.MjvCamera()
mujoco.mjv_defaultFreeCamera(leap_hand.m, camera1)

# Set camera position
camera1.lookat[:] = np.array([0.0, 0.0, 0.0])  # Point in space the camera is looking at
camera1.azimuth = -90                # Horizontal angle in degrees
camera1.elevation = -10                      # Vertical angle in degrees
camera1.distance = 1                         # Distance from the lookat point

camera2 = mujoco.MjvCamera()
mujoco.mjv_defaultFreeCamera(leap_hand.m, camera2)

# Set camera position
camera2.lookat[:] = np.array([0.0, 0.0, 0.0])  # Point in space the camera is looking at
camera2.azimuth = 0               # Horizontal angle in degrees
camera2.elevation = -90                      # Vertical angle in degrees
camera2.distance = 1                         # Distance from the lookat point

camera3 = mujoco.MjvCamera()
mujoco.mjv_defaultFreeCamera(leap_hand.m, camera3)

# Set camera position
camera3.lookat[:] = np.array([0.0, 0.0, 0.15])  # Point in space the camera is looking at
camera3.azimuth = 90               # Horizontal angle in degrees
camera3.elevation = -15                      # Vertical angle in degrees
camera3.distance = 1                         # Distance from the lookat point


In [4]:
mujoco.mj_step(leap_hand.m,leap_hand.d)

pos_worldi1=leap_hand.d.site(leap_hand.m.site('contact_index1').id).xpos.reshape(3)
rot_worldi1=leap_hand.d.site(leap_hand.m.site('contact_index1').id).xmat.reshape(3,3)
pos_worldt1=leap_hand.d.site(leap_hand.m.site('contact_thumb1').id).xpos.reshape(3)
rot_worldt1=leap_hand.d.site(leap_hand.m.site('contact_thumb1').id).xmat.reshape(3,3)

pos_worldi2=leap_hand.d.site(leap_hand.m.site('contact_index2').id).xpos.reshape(3)
rot_worldi2=leap_hand.d.site(leap_hand.m.site('contact_index2').id).xmat.reshape(3,3)
pos_worldt2=leap_hand.d.site(leap_hand.m.site('contact_thumb2').id).xpos.reshape(3)
rot_worldt2=leap_hand.d.site(leap_hand.m.site('contact_thumb2').id).xmat.reshape(3,3)

pos_world_palm=leap_hand.d.body(leap_hand.m.body('palm').id).xpos.reshape(3)
rot_world_palm=np.reshape(leap_hand.d.body(leap_hand.m.body('palm').id).xmat,[3,3])

pos_palmi1=pos_worldi1-pos_world_palm
rot_palmi1=rot_world_palm.T @ rot_worldi1
pos_palmt1=pos_worldt1-pos_world_palm
rot_palmt1=rot_world_palm.T @ rot_worldt1

pos_palmi2=pos_worldi2-pos_world_palm
rot_palmi2=rot_world_palm.T @ rot_worldi2
pos_palmt2=pos_worldt2-pos_world_palm
rot_palmt2=rot_world_palm.T @ rot_worldt2

# Rpk_index = np.reshape(leap_hand.d.body(leap_hand.m.body('mcp_joint').id).xmat,[3,3])

# Rpk_thumb = np.reshape(leap_hand.d.body(leap_hand.m.body('pip_4').id).xmat,[3,3])

Rpk_index=np.eye(3)
Rpk_thumb=np.eye(3)

qd_d=leap_hand.d
qd_m=leap_hand.m

# Capture initial pose of the 'cube' body
cube_id = leap_hand.m.body('cube').id

# Correct way to access the position and quaternion
initial_pos = leap_hand.d.xpos[cube_id].copy()  # Initial position [x, y, z]
initial_quat = leap_hand.d.xquat[cube_id].copy()  # Initial quaternion [x, y, z, w]

# Store the model reference separately (optional, if needed)
qd_m = leap_hand.m

print(f"Initial Position: {initial_pos}, Initial Quaternion: {initial_quat}")



Initial Position: [-0.     0.05  -0.135], Initial Quaternion: [1. 0. 0. 0.]


In [5]:
# result_index1=IK_index.calculate(pos_palmi1,'contact_index')
# result_thumb1=IK_thumb.calculate(pos_palmt1,'contact_thumb')
# result_index2=IK_index.calculate(pos_palmi2, 'contact_index')
# result_thumb2=IK_thumb.calculate(pos_palmt2, 'contact_thumb')

# print("result_index1",result_index1)
# print("result_thumb1",result_thumb1)
# print("result_index2",result_index2)
# print("result_thumb2",result_thumb2)


# framerate=30

# #go near the contact point
# while leap_hand.d.time < 2:

    

#     #placeholder code 
#     #index finger
#     [a,b,c,d]=result_index1
#     [e,f,g,h]=result_thumb1
    
#     leap_hand.apply_controls_hand([a,b,c,d,0,0,0,0,0,0,0,0,e,f,g,h])
    
#     #temperory
#     leap_hand.step_video(framerate,camera1)

# while leap_hand.d.time < 4 and leap_hand.d.time > 2:

    

#     #placeholder code 
#     #index finger
#     [a,b,c,d]=result_index2
#     [e,f,g,h]=result_thumb2
    
#     leap_hand.apply_controls_hand([a,b,c,d,0,0,0,0,0,0,0,0,e,f,g,h])
    
#     #temperory
    
#     leap_hand.step_video(framerate,camera1)

# leap_hand.play_video(framerate)

In [6]:
# Rpk_index=Rpk_index
# Rpk_thumb=Rpk_thumb

# b1 = np.array([0.028, 0, 0]).reshape(3, 1)
# b2 = np.array([-0.028, 0, 0]).reshape(3, 1)

# index_path='/home/iitgn-robotics/Saniya/redundancy-leap/leap-mujoco/model/leap hand/index_finger.xml'
# thumb_path='/home/iitgn-robotics/Saniya/redundancy-leap/leap-mujoco/model/leap hand/thumb.xml'


# # pos_worldi2=leap_hand.d.site(leap_hand.m.site('contact_index2').id).xpos.reshape(3)
# # rot_worldi2=leap_hand.d.site(leap_hand.m.site('contact_index2').id).xmat.reshape(3,3)

# while leap_hand.d.time < 10 and leap_hand.d.time > 4:
#     grasp = GraspClass()

#     n = 2
#     contact1 = leap_hand.d.site(leap_hand.m.site('contact_index2').id).xmat.reshape(3,3)
#     contact2 = leap_hand.d.site(leap_hand.m.site('contact_thumb2').id).xmat.reshape(3,3)

#     #from mujoco xml file


#     J1 = index_J=grasp.J(index_path,'contact_index',leap_hand.d.qpos[-16:-12])
#     J2 = index_J=grasp.J(index_path,'contact_index',leap_hand.d.qpos[-4:])

#     contact_orientations = [contact1, contact2]
#     Rpks = [Rpk_index, Rpk_thumb]
#     Js = [J1, J2]
#     r_theta = leap_hand.d.body(leap_hand.m.body('cube').id).xmat.reshape(3,3)
#     bs = [b1, b2]

#     # Compute the grasp matrices
#     Jh_ = grasp.Jh_full(n, contact_orientations, Rpks, Js)
#     G_ = grasp.G_full(n, contact_orientations, r_theta, bs)

#     H=grasp.selection_matrix(n,'SF')

#     Jh_leap=H@Jh_
#     G_leap_T=H@G_.T
#     G_leap=G_leap_T.T

#     # Controller parameters
#     Kp_d = 0.1*np.eye(6)
#     Kp_k = 1

#     n0 = 1*np.ones([8,1])
#     I = np.eye(8)
#     phi_d = np.ones([8,1])

# #leap_hand = LeapNode_Taucontrol()

# # Main control loop
#     q_d = leap_hand.d
#     q_m = leap_hand.m

#     posrot=PosRot()

#     q_final=posrot.q_subs(qd_m,q_m,qd_d,q_d,'cube')

#     # Compute forces
#     Fimp = np.dot(np.linalg.pinv(G_leap),np.dot(Kp_d , (q_final.reshape(6,1))))
#     Fnull = (I - np.matmul(np.linalg.pinv(G_leap), G_leap)) @ n0
    
#     # Compute desired torque
#     Tau_dy = Jh_leap.T @ (Fimp + Fnull)
    
    
    
#     # Read position from the Leap hand
#     #phi = leap_hand.read_pos()
#     phi=np.ones([8,1])
    
#     # Kinematic control torque
#     Tau_kin = Kp_k * (phi_d - phi)  # Corrected this line
    
#     # Total torque
#     Tau = Tau_dy + Tau_kin*0

#     leap_hand.d.ctrl[0:4]=Tau[0:4].reshape(4)
#     leap_hand.d.ctrl[-4:]=Tau[-4:].reshape(4)

    
#     leap_hand.step_video(framerate,camera1)
#     print("Current time:", leap_hand.d.time) 

# leap_hand.play_video(framerate)


In [6]:
Rpk_index = Rpk_index
Rpk_thumb = Rpk_thumb

b1 = np.array([0.028, 0, 0]).reshape(3, 1)
b2 = np.array([-0.028, 0, 0]).reshape(3, 1)

index_path = '/home/iitgn-robotics/Saniya/redundancy-leap/leap-mujoco/model/leap hand/redundancy/0_index_sim.xml'
thumb_path = '/home/iitgn-robotics/Saniya/redundancy-leap/leap-mujoco/model/leap hand/redundancy/0_thumb_sim.xml'

# Perform inverse kinematics calculations
result_index1 = IK_index.calculate(pos_palmi1, 'contact_index')
result_thumb1 = IK_thumb.calculate(pos_palmt1, 'contact_thumb_end')
result_index2 = IK_index.calculate(pos_palmi2, 'contact_index')
result_thumb2 = IK_thumb.calculate(pos_palmt2, 'contact_thumb_end')

print("result_index1", result_index1)
print("result_thumb1", result_thumb1)
print("result_index2", result_index2)
print("result_thumb2", result_thumb2)

framerate = 30  # Set video frame rate
simulation_duration = 120  # Total duration in seconds
start_time = time.time()  # Record start time

# Launch the MuJoCo viewer and run the simulation
with mujoco.viewer.launch_passive(leap_hand.m, leap_hand.d) as viewer:
    while time.time() - start_time < simulation_duration:
        current_time = time.time() - start_time  # Track simulation time

        # Phase 1: Move close to the contact points (first 2 seconds)
        if 1 < current_time < 3:
            [a, b, c, d] = result_index1
            [e, f, g, h] = result_thumb1
         
            leap_hand.apply_controls_hand([a, b, c, d, 0, 0, 0, 0, 0, 0, 0, 0, e, f, g, h])
            #leap_hand.d.qpos[-16:]=[a, b, c, d, 0, 0, 0, 0, 0, 0, 0, 0, e, f, g, h]

        # Phase 2: Continue to refine the contact (2 to 4 seconds)
        elif 3 < current_time < 8:
            [a, b, c, d] = result_index2
            [e, f, g, h] = result_thumb2
            
            leap_hand.apply_controls_hand([a, b, c, d, 0, 0, 0, 0, 0, 0, 0, 0, e, f, g, h])
            #leap_hand.d.qpos[-16:]=[a, b, c, d, 0, 0, 0, 0, 0, 0, 0, 0, e, f, g, h]

        # Phase 3: Execute logic between 4 and 10 seconds
        elif 8 < current_time < simulation_duration:
            grasp = GraspClass()
            n = 2

            contact1 = leap_hand.d.site(leap_hand.m.site('contact_index2').id).xmat.reshape(3, 3)
            contact2 = leap_hand.d.site(leap_hand.m.site('contact_thumb2').id).xmat.reshape(3, 3)

            J1 = grasp.J(index_path, 'contact_index', leap_hand.d.qpos[-16:-12])
            J2 = grasp.J(thumb_path, 'contact_thumb_end', leap_hand.d.qpos[-4:])

            contact_orientations = [contact1, contact2]
            Rpks = [Rpk_index, Rpk_thumb]
            Js = [J1, J2]
            r_theta = leap_hand.d.body(leap_hand.m.body('cube').id).xmat.reshape(3, 3)
            bs = [b1, b2]

            # Compute grasp matrices
            Jh_ = grasp.Jh_full(n, contact_orientations, Rpks, Js)
            G_ = grasp.G_full(n, contact_orientations, r_theta, bs)

            H = grasp.selection_matrix(n, 'HF')

            Jh_leap = H @ Jh_
            G_leap_T = H @ G_.T
            G_leap = G_leap_T.T

            # Controller parameters
            Kp_p_d = 1 * np.eye(3)
            Kp_o_d = 0.1 * np.eye(3)
            Kp_d = block_diag(Kp_p_d,Kp_o_d)
            Kp_k = 1

            n0 = 1*np.ones([6, 1])
            I = np.eye(6)
            phi_d = np.ones([6, 1])

            # Compute q_final
            # posrot = PosRot()
            # q_final = posrot.q_subs(qd_m,leap_hand.m, qd_d, leap_hand.d, 'cube')
            # print(q_final)

            # Inside the simulation loop, after capturing the current time:
            posrot = PosRot()
            q_final = posrot.q_subs(initial_pos, initial_quat, leap_hand.m, leap_hand.d, 'cube')
            # print(f"Pose Error (q_final): {q_final}")


            # posrot = PosRot()
            # q_final = posrot.q_subs(leap_hand.m, leap_hand.d, 'cube')

            if 15 < current_time < 16:
                cube_id = leap_hand.m.body('cube').id
                # Define the force to apply (e.g., 10 N along the X-axis)
                force = np.array([0, 0, 0])  # Force: [fx, fy, fz]
                torque = np.array([0.0, 0, 0.0])  # Torque: [τx, τy, τz]

                # Combine force and torque into a single array
                force_torque = np.hstack((force, torque))

                # Apply the force and torque to the body during simulation
                leap_hand.d.xfrc_applied[cube_id] = force_torque

            else: 
                cube_id = leap_hand.m.body('cube').id
                # Define the force to apply (e.g., 10 N along the X-axis)
                force = np.array([0, 0, 0])  # Force: [fx, fy, fz]
                torque = np.array([0.0, 0, 0.0])  # Torque: [τx, τy, τz]

                # Combine force and torque into a single array
                force_torque = np.hstack((force, torque))

                # Apply the force and torque to the body during simulation
                leap_hand.d.xfrc_applied[cube_id] = force_torque

            # Compute forces
            Fimp = np.dot(np.linalg.pinv(G_leap), np.dot(Kp_d, q_final.reshape(6, 1)))
            Fnull = (I - np.matmul(np.linalg.pinv(G_leap), G_leap)) @ n0
            #print(Fnull)

            # Compute desired torque
            Tau_dy = Jh_leap.T @ (Fimp*0+Fnull)
            # print('tau',Tau_dy)

            # Read position from the Leap hand
            phi = np.ones([8, 1])

            # Kinematic control torque
            # Tau_kin = Kp_k * (phi_d - phi)

            # Total torque
            Tau = Tau_dy # + Tau_kin * 0
            print(Tau)

            # Apply the computed torques
            leap_hand.d.ctrl[0:4] = Tau[0:4].reshape(4)
            leap_hand.d.ctrl[-4:] = Tau[-4:].reshape(4)

        # Step the simulation without camera-specific logic
        mujoco.mj_step(leap_hand.m, leap_hand.d)
        viewer.sync()  # Sync the viewer to reflect state updates

        # Print the current time for debugging
        # print("Current time:", current_time)

# Play the recorded video after the simulation ends
# leap_hand.play_video(framerate)


result_index1 [ 0.9215866  -0.04031082  0.60070308  0.35299859]
result_thumb1 [ 1.45621647  0.08100795 -0.44437575  0.56829299]
result_index2 [ 0.99900366 -0.04424744  0.64523576  0.37511287]
result_thumb2 [ 1.44023458  0.177796   -0.30272273  0.54653888]
[[ 0.12003985]
 [-0.00108442]
 [ 0.07585954]
 [ 0.03981554]
 [ 0.00071459]
 [-0.00223015]
 [ 0.10214654]
 [ 0.05787074]]
[[ 0.12003374]
 [-0.00108496]
 [ 0.07584875]
 [ 0.03980467]
 [ 0.00071425]
 [-0.00223619]
 [ 0.10213883]
 [ 0.05785959]]
[[ 0.12041084]
 [-0.00109308]
 [ 0.07617002]
 [ 0.04000549]
 [ 0.00076573]
 [-0.00224896]
 [ 0.10241364]
 [ 0.05794354]]
[[ 0.12078019]
 [-0.0011005 ]
 [ 0.07648131]
 [ 0.04019808]
 [ 0.00081688]
 [-0.00226509]
 [ 0.10267939]
 [ 0.05801863]]
[[ 0.1211427 ]
 [-0.00110737]
 [ 0.07678369]
 [ 0.04038339]
 [ 0.00086767]
 [-0.00228378]
 [ 0.10293686]
 [ 0.0580861 ]]
[[ 0.12149892]
 [-0.00111379]
 [ 0.07707786]
 [ 0.04056205]
 [ 0.00091808]
 [-0.00230443]
 [ 0.10318665]
 [ 0.05814692]]
[[ 0.12185028]
 [-